In [1]:
!wget https://www.dropbox.com/scl/fi/em9gg01fixe0evr2gm1yr/train.zip?rlkey=82gu4jxpee8ud3xfhkcwobu9l&dl=0 # train
!wget https://www.dropbox.com/scl/fi/toycvi2k3h0dz42gl3vk0/test.zip?rlkey=6wjn1yvhcfiuymbhkc2ubcro7&dl=0 # test
!wget https://www.dropbox.com/scl/fi/khtr33xeykwpjjezlbkdd/validation.zip?rlkey=zbpzzxv3c368df70nrhssi3sz&dl=0 # validation

--2024-03-15 17:48:07--  https://www.dropbox.com/scl/fi/em9gg01fixe0evr2gm1yr/train.zip?rlkey=82gu4jxpee8ud3xfhkcwobu9l
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc73e279cac1f63200a10cc6f7a6.dl.dropboxusercontent.com/cd/0/inline/CPJ5vRPPLHCcalKIMnVH5geCFeQftUzWB0oKiq0pWej75Zesu-g4hCIPPK-ossSYkSatuztH3DIUtiv6JW2BZO1T7ZwOe3RxQPtTuwLj8-CPalMdZFmsk967n0ReIjRbinE2RweCs9zS1DmnKy6z-KQA/file# [following]
--2024-03-15 17:48:08--  https://uc73e279cac1f63200a10cc6f7a6.dl.dropboxusercontent.com/cd/0/inline/CPJ5vRPPLHCcalKIMnVH5geCFeQftUzWB0oKiq0pWej75Zesu-g4hCIPPK-ossSYkSatuztH3DIUtiv6JW2BZO1T7ZwOe3RxQPtTuwLj8-CPalMdZFmsk967n0ReIjRbinE2RweCs9zS1DmnKy6z-KQA/file
Resolving uc73e279cac1f63200a10cc6f7a6.dl.dropboxusercontent.com (uc73e279cac1f63200a10cc6f7a6.dl.dropboxusercontent.com)... 162.125.80.15, 2620:

In [ ]:
# unzip the downloaded zip files
!unzip /content/train.zip?rlkey=82gu4jxpee8ud3xfhkcwobu9l # train
!unzip /content/test.zip?rlkey=6wjn1yvhcfiuymbhkc2ubcro7 # test
!unzip /content/validation.zip?rlkey=zbpzzxv3c368df70nrhssi3sz # validation

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
import os
total_train = sum([len(files) for r, d, files in os.walk(train_dir)]) - 1
total_test = sum([len(files) for r, d, files in os.walk(test_dir)]) - 1
total_valid = sum([len(files) for r, d, files in os.walk(valid_dir)]) - 1

In [12]:
train_dir = '/content/train'
test_dir = '/content/test'
valid_dir = '/content/validation'

img_width = 224
img_height = 224
color_channels = 3

input_shape = (img_width, img_height, color_channels)

image_shape=(img_width, img_height)
batch_size = 32
steps_per_epoch = total_train//32

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20,
                                   zoom_range=0.15, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.15,
                                   horizontal_flip=True, fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20,
                                   zoom_range=0.15, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.15,
                                   horizontal_flip=True, fill_mode='nearest')

In [11]:
train_gen = train_datagen.flow_from_directory(train_dir, target_size=image_shape,
                                              batch_size=batch_size,
                                              class_mode='categorical')

valid_gen = valid_datagen.flow_from_directory(valid_dir, target_size=image_shape,
                                              batch_size=batch_size,
                                              class_mode='categorical')

test_gen = train_datagen.flow_from_directory(test_dir,
                                             target_size=image_shape,
                                             batch_size=1,
                                             class_mode='categorical')

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [13]:
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=input_shape)

43941136/43941136 [==============================] - 2s 0us/step


In [14]:
for layer in base_model.layers:
    layer.trainable = False

In [15]:
model = Sequential()
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(
      train_gen,
      steps_per_epoch=steps_per_epoch,
      epochs=10,
      validation_data=valid_gen,
      validation_steps=total_valid//32)

Epoch 1/10
 7/37 [====>.........................] - ETA: 2:52 - loss: 1.4960 - accuracy: 0.3616